# Holder Wallet Conformance Flows (v3) - CTWalletCrossPreAuthorised

# 0.0 Initial setup

## 0.1 Setup conformance

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [2]:
from eudi_wallet.did_key import KeyDid, PublicKeyJWK
import uuid

# generate crypto seed
crypto_seed = b'helloworld'

key_did = KeyDid(seed=crypto_seed)
# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9KbptQwoaj2VP1S6Ahzo7REFCT4NBTPYdQinCZbCcyoqWKi9Q2uEW36DNSXhCwiYnGz6BAZkzytQAEBE5cPidCGnadH4SsLDbSZeG2SEChrqvQpdK4Mk8H32vs3B5g8Wr7kcc


## 1.1 Initiate Credential Issuance

In [3]:
from eudi_wallet.util import parse_query_string_parameters_from_url
from eudi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type = CredentialTypes.CTWalletCrossPreAuthorised
# qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
qr_code_data = "openid-credential-offer://?credential_offer_uri=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fissuer-mock%2Foffers%2F674c272c-4d21-47d7-9190-2fd26ebc56bb"
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)

[18:48:47] Credential offer URI:                                                                   ]8;id=902424;file:///tmp/ipykernel_73016/2045094697.py\2045094697.py]8;;\:]8;id=704441;file:///tmp/ipykernel_73016/2045094697.py#16\16]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/674c272c-4d21-47d7-91                 
           90-2fd26ebc56bb                                                                                         

{'credential_issuer': 'https://api-conformance.ebsi.eu/conformance/v3/issuer-mock', 'credentials': [{'format': 'jwt_vc', 'types': ['VerifiableCredential', 'VerifiableAttestation', 'CTWalletCrossPreAuthorised'], 'trust_framework': {'name': 'ebsi', 'type': 'Accreditation', 'uri': 'TIR link towards accreditation'}}], 'grants': {'urn:ietf:params:oauth:grant-type:pre-authorized_code': {'pre-authorized_code': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW95dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpOVEydUVXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRLNE1rOEgzMnZzM0I1ZzhXcjdrY2MiLCJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlma

[18:48:48] Credential offer:                                                                       ]8;id=290687;file:///tmp/ipykernel_73016/2045094697.py\2045094697.py]8;;\:]8;id=676385;file:///tmp/ipykernel_73016/2045094697.py#19\19]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletCrossPreAuthorised'                                                            
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'urn:ietf:params:oauth:grant-type:pre-authorized_code': {                                       
                       'pre-authorized_code':                                                                      
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNF                 
           ZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1                 
           BnWW95dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpO                 
           VEydUVXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRL                 
           NE1rOEgzMnZzM0I1ZzhXcjdrY2MiLCJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3J                 
           lZGVudGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS                 
           5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50a                 
           WFsIiwiVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc1ByZUF1dGhvcmlzZWQiXX1dLCJpYXQi                 
           OjE2ODg0NzY2OTMsImV4cCI6MTY4ODQ3Njk5MywiaXNzIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5                 
           ldS9jb25mb3JtYW5jZS92My9pc3N1ZXItbW9jayIsImF1ZCI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2                 
           kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pid                 
           XVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJD                 
           Y3lvcVdLaTlRMnVFVzM2RE5TWGhDd2

[18:48:49] Credential issuer configuration:                                                        ]8;id=733360;file:///tmp/ipykernel_73016/2045094697.py\2045094697.py]8;;\:]8;id=171905;file:///tmp/ipykernel_73016/2045094697.py#22\22]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cre                 
           dential',                                                                                               
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer                 
           -mock/credential_deferred',                                                                             
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework':

           Authorization server configuration:                                                     ]8;id=196321;file:///tmp/ipykernel_73016/2045094697.py\2045094697.py]8;;\:]8;id=849332;file:///tmp/ipykernel_73016/2045094697.py#25\25]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/au                 
           thorize',                                                                                               
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Exchange code for access token

In [4]:
from eudi_wallet.siop_auth.util import exchange_pre_authorized_code_for_access_token

token_uri = auth_server_configuration.token_endpoint
user_pin = "5106"
access_token_response = await exchange_pre_authorized_code_for_access_token(token_uri, user_pin, credential_offer.grants.get("urn:ietf:params:oauth:grant-type:pre-authorized_code").get("pre-authorized_code"))
console.log("Access token response: ", access_token_response)

{'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjVmZTllZDA3LWJmYjQtNGQ4OC04NjM0LWY0Y2QyNGM1YmVhNiIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc1ByZUF1dGhvcmlzZWQiXX1dLCJjX25vbmNlIjoiN2NjZTBlZmMtNThjNS00MWYwLTk5OGEtZjlmZGNlODdlNjE5IiwiY19ub25jZV9leHBpcmVzX2luIjo4NjQwMCwiY2xpZW50X2lkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXRRQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIn0sImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2NrIiwiYXVkIjpbImh0dHBzOi8vYXBpLWN

[18:48:50] Access token response:                                                                   ]8;id=230373;file:///tmp/ipykernel_73016/3134750599.py\3134750599.py]8;;\:]8;id=390116;file:///tmp/ipykernel_73016/3134750599.py#6\6]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYW                
           VZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjVmZTllZDA3LWJmYjQtNGQ4OC04NjM0LWY0Y2Q                
           yNGM1YmVhNiIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGV                
           udGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnN                
           pLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc1ByZUF1dGhvcmlzZWQiXX1dLCJjX25vbmNlIjo                
           iN2NjZTBlZmMtNThjNS00MWYwLTk5OGEtZjlmZGNlODdlNjE5IiwiY19ub25jZV9leHBpcmVzX2luIjo4NjQwMCw                
           iY2xpZW50X2lkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo                
           5S2JwdFF3b2FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd                
           6NkJBWmt6eXRRQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2N                
           jIn0sImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2N                
           rIiwiYXVkIjpbImh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvaXNzdWVyLW1                
           vY2siXSwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo                
           5S2JwdFF3b2FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd                
           6NkJBWmt6eXRRQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2N                
           jIiwiaWF0IjoxNjg4NDc2NzMwLCJleHAiOjE2ODg1NjMxMzB9.6w4XDA-YPSL5LOUtT-Rapc8Ba3ahDsgq1T6YCs                
           AWTFSBrJQP8bI7vkBV6ed5ZPmVkHjpYoa4Ztby2HTUceOlXw',                                                      
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUW                
           lGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6ImU2YmIwNmZlLTNlYjYtNGQ0Yy05NTNmLWQyYTJkMTJ                
           hN2MwYiIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1                
           tb2NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo                
           5S2JwdFF3b2FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd                
           6NkJBWmt6eXRRQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2N                
           jIiwiYXVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J                
           wdFF3b2FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJ                
           BWmt6eXRRQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiw                
           iaWF0IjoxNjg4NDc2NzMwLCJleHAiOjE2ODg0NzY4NTB9.aXuGf9jgkAuWtZpwtvAyR_Wi5-trxJdGN1MY6ZMh6L                
           nUUmWVExWx5HC4fjIKUkImbppMp4szpFazzqxo-8N_rg',                                                          
               c_nonce='7cce0efc-58c5-41f0-998a-f9fdce87e619',                                                     
               c_nonce_expires_in=86400   

## 1.3 Request credential (cross device)

In [5]:
from eudi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
credential_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
print("Credential response: ", credential_response)

           Credential request JWT:                                                                   ]8;id=608975;file:///tmp/ipykernel_73016/621865142.py\621865142.py]8;;\:]8;id=871512;file:///tmp/ipykernel_73016/621865142.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVW               
           jNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1               
           hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjV               
           nOFdyN2tjYyN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2FqMlZQ               
           MVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXRRQUVCR               
           TVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwidHlwIjoib3Blbm               
           lkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1h               
           bmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjg4NTYzMTMwLCJpYXQiOjE2ODg0NzY3MzAsImlzcyI6ImRpZDprZ               
           Xk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem               
           83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZEN               
           HbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsIm5vbmNlIjoiN2NjZTBlZmMt               
           NThjNS00MWYwLTk5OGEtZjlmZGNlODdlNjE5In0.C_uz5t4WfAFBmH-bjN8XTW7poQJE_mTO0a9DCskKugK9CiWBc               
           LiTik7MHQrLgplLsUGZIl5Pogtj6bJ6f2-E5w                                                                   

Credential response:  {'format': 'jwt_vc', 'credential': 'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNvbmZvcm1hbmNlI2MxNzg4MGYzLTgzYjMtNGM5Mi05YmRhLTNiYzI3MzczNWRhMiIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsImlzcyI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjg4NDc2NzMwLCJleHAiOjE2ODg1NjMxMzAsImlhdCI6MTY4ODQ3NjczMCwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwiaWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlI2MxNzg4MGYzLTgzYjMtNGM5Mi05YmRhLTNiYzI3MzczNWRhMiIsInR5cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldENyb3NzUHJlQXV0aG9yaXNlZCJdLCJpc3N1ZXIiOiJkaWQ6ZWJzaTp6aEpBUmpQTjY5Y0V